# Decision Analysis - Project 1 #


Mateusz Małecki <br>
Daniel Jankowski 148257

In [5]:
import pandas as pd
import numpy as np
import math
from typing import Optional


In [6]:
data = pd.read_csv("DA_database.csv", decimal=",")

In [7]:
data.head()

,name,prestige,power,price,engine_size,colour_preference,fuel_consumption,mileage,production_year,automatic_gear_box
0,Seat Ibiza 1.2 white,1,90,34900,1.2,2,6.0,90000,2015,0
1,Seat Ibiza 1.4 green,1,85,27800,1.4,4,6.9,82000,2015,0
2,Skoda Fabia grey,4,75,31500,1.0,1,5.8,215000,2019,0
3,Nissan Note grey,3,80,28500,1.2,1,6.0,133000,2014,0
4,MINI Cooper 1.6 blue,7,175,27900,1.6,5,2.0,183000,2006,0


## Decision classes ##

In [8]:
params={
    "prestige":{
    "type":"gain", "p":2, "q":1, "v":None, "w":5
    },
    "power":{
    "type":"gain", "p":15, "q":5, "v":None, "w":7
    },
    "price":{
    "type":"cost", "p":3000, "q":1000, "v":None, "w":8
    },
    "engine_size":{
    "type":"cost", "p":0.2, "q":0, "v":None, "w":7
    },
    "colour_preference":{
    "type":"gain", "p":0, "q":0, "v":None, "w":2
    },
    "fuel_consumption":{
    "type":"cost", "p":0.3, "q":0.1, "v":None, "w":8
    },
    "mileage":{
    "type":"cost", "p":35000, "q":20000, "v":None, "w":10
    },
    "production_year":{
    "type": "gain", "p":4, "q":2, "v":None, "w":6
    },
    "automatic_gear_box":{
    "type":"gain", "p":1, "q":0, "v":None, "w":3
    },
}

## Promethe ##

In [ ]:
class Promethee:
    def __init__(self, alternatives: pd.DataFrame, parameters: dict[str, dict]) -> None:
        self.alternatives = alternatives
        self.parameters = parameters
        self.criteria = [x for x in alternatives.columns[1:]]


    def criteriaComparison(self, criteria: str, a, b, funcType: str) -> float:
        assert(funcType in ["Type1", "Type2", "Type3", "Type4", "Type5", "Type6"])
        p, q = self.parameters[criteria]["p"], self.parameters[criteria]["q"]
        diff = a - b if self.parameters[criteria]["type"] == "gain" else b-a
        if funcType == "Type1":
            return 1 if diff > 0 else 0
        
        elif funcType == "Type2":
            return 1 if diff > q else 0
        
        elif funcType == "Type3":
            if diff > p:
                return 1
            elif diff <= 0:
                return 0
            else:
                return diff / p
        
        elif funcType == "Type4":
            if diff > p:
                return 1
            elif diff < q:
                return 0
            else:
                return 0.5
        
        elif funcType == "Type5":
            if diff > p:
                return 1
            elif diff <= q:
                return 0
            else:
                return (diff - q)/(p-q)
        else:
            return 0 if diff <= 0 else 1 - math.exp(-diff/2) # assuming sd = 1
    
    def computeMarginalPreferencesAllCriteria(self, funcType: str) -> np.array:
        marginalPreferenceIndices = np.zeros((len(self.criteria), len(self.alternatives), len(self.alternatives)))
        for n, criteria in enumerate(self.criteria):
            for i, row in self.alternatives.iterrows():
                for j, row2 in self.alternatives.iterrows():
                    if i == j:
                        continue
                    marginalPreferenceIndices[n][i][j] = self.criteriaComparison(criteria, row[criteria], row2[criteria], funcType)
        return marginalPreferenceIndices
    
    def computeComprehensiveMatrix(self, funcType: str) -> np.array:
        print(np.array([x['w'] for x in self.parameters.values()]))
        return np.sum(self.computeMarginalPreferencesAllCriteria(funcType) * np.array([x['w'] for x in self.parameters.values()]), axis=0)
    
    def computePositiveFlows(self, matrix) -> np.array:
        return np.sum(matrix, axis = 1)
    def computeNegativeFlows(self, matrix) -> np.array:
        return np.sum(matrix, axis = 0)
    
    def prometheeI(self, preference_function: str = "Type5"):
        X = self.computeComprehensiveMatrix(preference_function)
        positive_flows = sorted([x for x in zip(self.alternatives["name"], self.computePositiveFlows(X))], key=lambda x:x[1], reverse=True)
        negative_flows = sorted([x for x in zip(self.alternatives["name"], self.computeNegativeFlows(X))], key=lambda x:x[1])
        print(*positive_flows)
        print(*negative_flows)
        # graph = []
        # pos = positive_flows[0]
        # neg = negative_flows[0]
        # for i in range(1, len(positive_flows)):
        #     next_pos = positive_flows[i]
        #     next_neg = negative_flows[i]



In [ ]:
decisionMaker = Promethee(data, params)
decisionMaker.prometheeI()